
# Генерация синтетических данных 

Для выполнения ТЗ на позицию Data-analyst предлагается использовать таблицы со следующей структурой данных:

*	Таблица "users" с полями: id, name, email, created_at

*	Таблица "orders" с полями: id, user_id, total_price, created_at

*	Таблица "order_items" с полями: id, order_id, product_name, price, quantity


In [1]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import datetime as dt

In [2]:
# Используя метод Faker() из библиотеки faker сгенерируем тестовые данные.
fake = Faker()

n = 2500000 # объявим количество генерируемых значений   

# Создадим пустой список 'names' и с помощью цикла заполним его генерируемыми значениями.
names=[]
for i in range(n):
    names.append(fake.name())

# names = [fake.name() for i in range(n)]

In [3]:
# Создадим пустой список 'dates', зададим период времени, в котором будут генерироваться даты,
# с помощью цикла заполним его генерируемыми значениями.
dates=[]
date_start = '2023-01-01'
date_start = dt.datetime.strptime(date_start, '%Y-%m-%d')
date_end = '2024-09-30'
date_end = dt.datetime.strptime(date_end, '%Y-%m-%d')

for i in range(n):
    dates.append(fake.date_between(date_start, date_end))

dates.sort()

In [4]:
# Создадим пустой список 'emails' и с помощью цикла заполним его.
# Чтобы значения были уникальными добавим перед  "@"  порядковый номер имени в списке names.
emails = [name.lower().replace(" ", ".") + f"{i}@googlyandex.com" for i, name in enumerate(names)]

In [5]:
# Cоздадим пустой DataFrame, куда и будем добавлять сгенерировынные данные. 
df_users = pd.DataFrame()

# Соберём DataFrame
df_users['name'] = names
df_users['email'] = emails
df_users['created_at'] = dates

df_users.head()

,name,email,created_at
0,Chelsey White,chelsey.white0@googlyandex.com,2023-01-01
1,Gregory Allen,gregory.allen1@googlyandex.com,2023-01-01
2,Rachel Jones,rachel.jones2@googlyandex.com,2023-01-01
3,James Evans,james.evans3@googlyandex.com,2023-01-01
4,Sarah Fisher,sarah.fisher4@googlyandex.com,2023-01-01


In [6]:
# Cбросим индекс и выведем его в отдельную колонку.
# Переименуем колонку 'index' в 'id'.
df_users = df_users.reset_index().rename(columns={'index': 'id'})

# Так как мы генерируем наши данные для Postgre SQL, учитывая ограничение первичного и внешнего ключей, а также
# разницу в исчислении в SQL и Python, сдвинем все значения столбца "id" на +1.
df_users['id'] = df_users['id'] + 1


print(df_users.head())
print('----------------------')
print(df_users.shape)
print('----------------------')
print(df_users.dtypes)

   id           name                           email  created_at
0   1  Chelsey White  chelsey.white0@googlyandex.com  2023-01-01
1   2  Gregory Allen  gregory.allen1@googlyandex.com  2023-01-01
2   3   Rachel Jones   rachel.jones2@googlyandex.com  2023-01-01
3   4    James Evans    james.evans3@googlyandex.com  2023-01-01
4   5   Sarah Fisher   sarah.fisher4@googlyandex.com  2023-01-01
----------------------
(2500000, 4)
----------------------
id             int64
name          object
email         object
created_at    object
dtype: object


________________________________________________________________________________________

Таблица "orders" с полями: 
* id, 
* user_id, 
* total_price, 
* created_at

In [7]:
# Таблицу 'orders' сформируем следующим образом:
# 1. ИЗ df_users случайным образом отберём n * 5 строк ['id','created_at']
# 2. К каждой дате добавим случайное количество дней из диапазона от 0 до 60
# 3. Общую стоимость сгенерируем случайным образом

df_orders = df_users[['id','created_at']].sample((n * 5), replace = True).rename(columns={'id': 'user_id'})

df_orders['created_at'] = df_orders.created_at + pd.Timedelta(days = (random.randint(0,60)))



print(df_orders.head(2))
print('----------------------')
print(df_orders.shape)

         user_id  created_at
453981    453982  2023-05-25
1924443  1924444  2024-06-03
----------------------
(12500000, 2)


In [8]:
# Соберём DataFrame
df_orders['total_price'] = np.nan
df_orders.head(2)

,user_id,created_at,total_price
453981,453982,2023-05-25,NaN
1924443,1924444,2024-06-03,NaN


In [9]:
df_orders = df_orders.sort_values('created_at', ascending=True).reset_index(drop=True)
df_orders = df_orders.reset_index().rename(columns={'index': 'order_id'})
df_orders['order_id'] = df_orders['order_id'] + 1
df_orders.head(2)

,order_id,user_id,created_at,total_price
0,1,2774,2023-01-29,NaN
1,2,2686,2023-01-29,NaN


____________________________________________________________________________________________________

По аналогии поступим и с последней таблицей.

Таблица "order_items" с полями: id, order_id, product_name, price, quantity

In [10]:
df_order_items = df_orders[['order_id','total_price']].copy()
df_order_items.head()

,order_id,total_price
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [11]:
product_name=[]
for i in range(int(n * 5)):
    product_name.append(fake.bothify(text='Prod.№: ????-########'))

df_order_items['product_name'] = product_name
df_order_items.head()    

,order_id,total_price,product_name
0,1,NaN,Prod.№: kDOe-12612877
1,2,NaN,Prod.№: jMCN-28203691
2,3,NaN,Prod.№: NqEu-83181983
3,4,NaN,Prod.№: ELjg-02215747
4,5,NaN,Prod.№: ZlNz-37570014


In [12]:
quantity=[]
for i in range(int(n * 5)):
    quantity.append(random.randrange(1, 100, 1))

df_order_items['quantity'] = quantity

In [13]:
# Создадим список 'prices' и заполним его псевдослучайными целыми числами от 5 до 500 с шагом 5
prices=[]
for i in range(int(n *5)):
     prices.append(random.randrange(5, 500, 5))

df_order_items['price'] = prices

In [14]:
df_order_items['total_price'] = df_order_items.quantity * df_order_items.price

In [15]:
df_order_items = df_order_items.reset_index(drop=True)
df_order_items = df_order_items.reset_index().rename(columns={'index': 'id'})
df_order_items['id'] = df_order_items['id'] + 1
df_order_items.head(2)

,id,order_id,total_price,product_name,quantity,price
0,1,1,18090,Prod.№: kDOe-12612877,67,270
1,2,2,1320,Prod.№: jMCN-28203691,8,165


____________________________________________________________________

In [16]:
# Сохраним DataFrame в файл.csv
# Так как колонка 'id' у нас будет являться первичным ключом, и мы при создании таблицы установим параметры для неё SERIAL PRIMARY KEY, избавимся от колонок 'id' в наших датафреймах.
df_users = df_users[['name', 'email', 'created_at']] 
df_users.to_csv('users.csv', index=False)

In [17]:
df_orders['total_price'] = df_order_items['total_price'].copy()
df_orders = df_orders[['user_id', 'created_at', 'total_price']]
df_orders.to_csv('orders.csv', index=False)

In [18]:
df_order_items = df_order_items[['order_id', 'product_name', 'price', 'quantity']]
df_order_items.to_csv('order_items.csv', index=False)